These files are not easy to use in their current chronological arrangement, though having them in a large system like GBQ will solve a lot of our problems. Nevertheless, it’ll be convenient to have a local sample of owners to do work. 
This task asks you to generate a file of owners where the file contains every record for each owner. There will be more than one owner in the file, and I do not want you to include card_no==3, which is the code for non-owners. The size of the sample is up to you, but I’d recommend shooting for a sample that’s around 250 MB. That’s big enough to be rich, but small enough to be fast. Ish.
Deliverable
A python script that handles the following tasks: 
1.	Connects to your GBQ instance.
2.	Builds a list of owners. 
3.	Takes a sample of the owners. 
4.	Extracts all records associated with those owners and writes them to a local text file. 
You’ll submit your code carrying out the steps. 


In [2]:
import pandas as pd
import numpy as np
import pandas_gbq
import sqlite3
import db_dtypes

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

In [4]:
# These first two values will be different on your machine. 
service_path = ""
service_file = 'umt-msba-17ad1778bcbd (2).json' # change this to your authentication information  

gbq_proj_id = 'umt-msba'  

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(private_key)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [5]:
Query ="""   
With rand_cte AS (
SELECT Distinct card_no
FROM `umt-msba.transactions.transArchive_*`
Where card_no != 3
ORDER BY RAND()
LIMIT 80
)

SELECT *
FROM `umt-msba.transactions.transArchive_*` AS transactions
JOIN rand_cte
On rand_cte.card_no = transactions.card_no

"""

In [7]:
query_job = client.query(Query)  # Start the query job
results = query_job.result()

In [8]:
df = client.query(Query).to_dataframe()

df.to_csv('owners.csv', index=False)